This code is part of the lab forum at AAAI-24 on "**Harnessing Large Language Models for Planning:** A Lab on Strategies for Success and Mitigation of Pitfalls."


---



In this code, we will see how to use Plansformer for inference.

In [1]:
!pip install transformers
!pip install sentencepiece

In [2]:
# Downloading Plansformer from Dropbox
!wget -O plansformer.zip https://www.dropbox.com/scl/fi/sc2syaax93i50bn14gn3w/plansformer_v3.zip?rlkey=a5qf2fanfn5028yx78drf1hic&dl=0

--2024-02-16 01:23:05--  https://www.dropbox.com/scl/fi/sc2syaax93i50bn14gn3w/plansformer_v3.zip?rlkey=a5qf2fanfn5028yx78drf1hic
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6025:18::a27d:4512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc7486f58f2bc7736aca929e5cf1.dl.dropboxusercontent.com/cd/0/inline/CNVvhuPYC9Zcy0g76CWbNmGSNheXLqnngCwbcaphjxQo_dw_SC2nPSiZFp8o8v1hmu0AXID-r-62VmpaXqCbVxflPKHNG9Z_poKrOK6g1pmfpANEJ9S9urK9-IaK0JV9jACRe3Z2ne7dnoqnhbcau_GW/file# [following]
--2024-02-16 01:23:06--  https://uc7486f58f2bc7736aca929e5cf1.dl.dropboxusercontent.com/cd/0/inline/CNVvhuPYC9Zcy0g76CWbNmGSNheXLqnngCwbcaphjxQo_dw_SC2nPSiZFp8o8v1hmu0AXID-r-62VmpaXqCbVxflPKHNG9Z_poKrOK6g1pmfpANEJ9S9urK9-IaK0JV9jACRe3Z2ne7dnoqnhbcau_GW/file
Resolving uc7486f58f2bc7736aca929e5cf1.dl.dropboxusercontent.com (uc7486f58f2bc7736aca929e5cf1.dl.dropboxusercontent.com)... 162.125.69.

In [3]:
!unzip plansformer.zip -d ./

Archive:  plansformer.zip
   creating: ./codet5-base/
  inflating: ./codet5-base/predictions.csv  
   creating: ./codet5-base/model_files/
  inflating: ./codet5-base/model_files/pytorch_model.bin  
  inflating: ./codet5-base/model_files/tokenizer_config.json  
  inflating: ./codet5-base/model_files/merges.txt  
  inflating: ./codet5-base/model_files/config.json  
  inflating: ./codet5-base/model_files/special_tokens_map.json  
  inflating: ./codet5-base/model_files/vocab.json  
  inflating: ./codet5-base/model_files/generation_config.json  
  inflating: ./codet5-base/logs.txt  


In [4]:
model_path = "codet5-base/model_files"

In [22]:
from transformers import RobertaTokenizer, T5ForConditionalGeneration
import torch
from torch import cuda
from IPython.display import display, HTML

device = 'cuda' if cuda.is_available() else 'cpu'

In [6]:
tokenizer = RobertaTokenizer.from_pretrained(model_path, local_files_only = True)
model = T5ForConditionalGeneration.from_pretrained(model_path, local_files_only = True)
model = model.to(device)

In [7]:
problem = """
<GOAL>ontable b1, clear b1, ontable b2, clear b2, ontable b3, clear b3, ontable b4, clear b4
<INIT>handempty, ontable b1, on b2 b3, clear b2, ontable b3, on b4 b1, clear b4
<ACTION> pick-up <PRE> clear x, ontable x, handempty <EFFECT> not ontable x, not clear x, not handempty, holding x
<ACTION> put-down <PRE> holding x <EFFECT> not holding x, clear x, handempty, ontable x
<ACTION> stack <PRE> holding x, clear y <EFFECT> not holding x, not clear y, clear x, handempty, on x y
<ACTION> unstack <PRE> on x y, clear x, handempty <EFFECT> holding x, clear y, not clear x, not handempty, not on x y"
"""

In [23]:
# Replacing markers with styled HTML spans for better visualization of the problem
formatted_problem = problem\
    .replace("<GOAL>", "<span class='goal'>GOAL:</span>")\
    .replace("<INIT>", "<br><span class='init'>INIT:</span>")\
    .replace("<ACTION>", "<br><span class='action'>ACTION:</span><br>")\
    .replace("<PRE>", "<span class='pre'>PRE:</span>")\
    .replace("<EFFECT>", "<span class='effect'>EFFECT:</span>")

# Adding CSS styles
styles = """
<style>
.goal { color: #f1c40f; font-weight: bold; }
.init { color: #16a085; font-weight: bold; }
.action { color: #2980b9; font-weight: bold; }
.pre { color: #8e44ad; font-weight: bold; }
.effect { color: #c0392b; font-weight: bold; }
span { margin-right: 5px; }
body { font-family: Arial, sans-serif; font-size: 14px; }
</style>
"""

# Display with HTML and styles
display(HTML(styles + f'<div>{formatted_problem}</div>'))

In [24]:
input_ids = tokenizer.encode(problem, return_tensors='pt').to(device, dtype = torch.long)
generated_ids = model.generate(
input_ids, num_beams = 2, max_length=input_ids.shape[-1] + 2,
repetition_penalty = 2.5, length_penalty = 1.0,
early_stopping=False)
predicted_plan = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

In [25]:
# Printing the plan generated by Plansformer for the given problem
predicted_plan

'unstack b2 b3, put-down b2, unstack b4 b1, put-down b4'